# Part 2: Segmenting and Clustering Neighborhoods in Toronto

## Pre-processing (sames as part 1)

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki_page= requests.get(wiki_link).text

soup = BeautifulSoup(raw_wiki_page,'lxml')
#print(soup.prettify())

In [3]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [4]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

## Part 2: Getting geographical coordinates of each postal code

In [11]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['Postcode', 'Latitude', 'Longitude']

### Creating Pandas Dataframe

In [14]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)

In [16]:
toronto_merge = pd.merge(df_toronto, df_latlng, on=['Postcode'], how='inner')

In [17]:
toronto_merge.to_csv('toronto_p2.csv')
toronto_merge.head(10)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Downtown Toronto,Commerce Court,M5L,43.648198,-79.379817
1,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
2,North York,Bedford Park,M5M,43.733283,-79.419750
3,East Toronto,India Bazaar,M4L,43.668999,-79.315572
4,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,43.739416,-79.588437
5,East York,Thorncliffe Park,M4H,43.705369,-79.349372
6,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
7,North York,CFB Toronto,M3K,43.737473,-79.464763
8,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
9,East Toronto,Riverdale,M4K,43.679557,-79.352188
